# Silicon - BGW/Inteqp

In this directory we calculate an interpolated quasiparticle bandstructure for
silicon using the Inteqp code from BerkeleyGW.

Follow these steps:

### Point 1

Run the inteqp calculation using `./01-run_inteqp.run`.

### Point 2

Study the input file `inteqp.inp` and compare to the input file description
   in `BSE/absorption.inp` (sections pertaining to inteqp).  Why do we need to
   interpolate rather than just directly run Sigma on these k-points? How is
   the number of coarse points listed determined? Why do we want to use
   symmetries on the coarse grid but not the fine one?

### Point 3

What files are linked into this directory? How are they used?

### Point 4

Use `degeneracy_check.x` to determine what numbers of conduction and valence
   bands are acceptable with respect to degeneracy from `WFN_co` and `WFN_fi`.
   Are the numbers listed in `inteqp.inp` acceptable? How many bands are
   available in `WFN_co` and `WFN_fi`? How do the degeneracy numbers compare to
   those needed for Epsilon or Sigma? Why?

### Point 5

When the job is finished, take a brief look at `inteqp.out`. What warning do
   you see at the end? Look at `dcmat_norm.dat` and `dvmat_norm.dat` to see
   which values are being flagged as less accurate. How could you improve the
   accuracy of the interpolation for these energy levels?

### Point 6
    
Plot the resulting bandstructure.

In [ ]:
import matplotlib.pyplot as plt
from plot_bandstructure import plot_bandstructure

plot_bandstructure()

### Point 7
        
Using `Si_bands.png` and `bandstructure.dat`, find the energy and location
   of the the direct gap and indirect gap, for LDA and GW. What is the valence
   band- width for LDA and GW? How do the band curvatures (effective masses)
   and band topology compare between LDA and GW?

### Point 8

Run the `qp_shifts.py` script on `../sigma_hp.log` to produce `cond` and `val` files
   containing a list of MF eigenvalues (column 1) and quasiparticle corrections
   (`E_qp - E_MF`, column 2) for conduction and valence bands, respectively, in a
   format suitable for plotting. Make a graph of quasiparticle corrections as a
   function of initial energy. Over what range of energies around the gap would
   a linear fit be appropriate? Why is the relation different for conduction
   and valence? Why is it non-linear farther away?

In [ ]:
import sys
from qp_shifts import qp_shifts
qp_shifts(infilename='../sigma_hp.log',valtop=4)   #valtop = index of VBM

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

val = np.loadtxt('val')
val_MF = val[:,0]
val_QP = val[:,1]
cond = np.loadtxt('cond')
cond_MF = cond[:,0]
cond_QP = cond[:,1]

fig = plt.figure(figsize=(10,8))
matplotlib.rc('font', size=14.0)
plt.plot(val_MF, val_QP, 'bo-', label='Valence band')
plt.plot(cond_MF, cond_QP, 'ro-', label='Conduction band')
plt.legend()
plt.xlabel('MF energies (eV)')
plt.ylabel('QP corrections (eV)')

#Uncomment to save the figure
# plt.savefig('QP_corrections.png')

### Point 9

Perform a linear fit on the linear regime, separately for `cond` and `val`.
   How good is the fit? How well does it describe the QP corrections farther from
   the gap? You will later use these fits in the calculation of the optical
   absorption, as a stretch goal in a later example.


In [ ]:
def fit_energy(E_min, E_max, x, y, label):
    Energy = np.linspace(E_min, E_max, 100)
    E_idx = np.logical_and(x>E_min, x<E_max)
    z = np.polyfit(x[E_idx], y[E_idx], 1)
    print('Slope:'+str(z[0]))
    y_fit = z[0]*x+z[1]
    plt.plot(x, y, 'bo-', label='Computed '+label)
    plt.plot(x, y_fit, 'r-', linewidth=2, label='Fit '+label)
    plt.legend()
    plt.xlabel('MF energies (eV)')
    plt.ylabel('QP corrections (eV)')

fig = plt.figure(figsize=(10,8))
#Valence bands
E_min = 3
E_max = 6
fit_energy(E_min, E_max, val_MF, val_QP, 'val')
#Conduction bands
E_min = 6
E_max = 9
fit_energy(E_min, E_max, cond_MF, cond_QP, 'cond')

#Uncomment to save the figure
# plt.savefig('QP_corrections_fit.png')

  ### Point 10

Do a fit also on the whole range of values. How good are the fits? Apply these linear relations to
    `bandstructure.dat` and compare the results to the more rigorous (but
    time-consuming) calculation from inteqp. You can also add the inteqp
    `bandstructure.dat` values to `cond` and `val` and see how the points there
    compare to the ones from Sigma on the regular grid.

In [ ]:
fig = plt.figure(figsize=(10,8))
#Valence bands
E_min = -6
E_max = 6
fit_energy(E_min, E_max, val_MF, val_QP, 'val')
#Conduction bands
E_min = 6
E_max = 23
fit_energy(E_min, E_max, cond_MF, cond_QP, 'cond')

#Uncomment to save the figure
# plt.savefig('QP_corrections_fitall.png')